In [1]:
%load_ext line_profiler
import line_profiler

In [2]:
import os 
import importlib
import sys
import tensorflow as tf
import itertools
import numpy as np
import numba as nb
from math import sqrt, log, exp
from numpy import unravel_index
from random import choice, random, sample
np.random.seed(1337)  # for reproducibility
from keras.models import Sequential, Model, load_model
import keras.backend as K
import matplotlib.pyplot as plt
K.set_image_dim_ordering('th')
import time

Using TensorFlow backend.


In [22]:
class Ataxx:
    def __init__(self, board=None):
        if board is None:                  # if there is no initialization given
            self.data = np.zeros((7, 7), dtype=np.int8)   # then generate a board with starting init, and black(-1) takes first turn
            self.data[0, 0] = -1           
            self.data[6, 6] = -1
            self.data[0, 6] = 1
            self.data[6, 0] = 1
        else:
            self.data = board.copy()
            
    def reset(self, board=None):
        if board is None:
            self.data = np.zeros((7, 7), dtype=np.int8)
            self.data[0, 0] = -1           
            self.data[6, 6] = -1
            self.data[0, 6] = 1
            self.data[6, 0] = 1
        else:
            self.data = board.copy()
        
    def get_feature_map(self, turn, move):
        out = np.zeros((6, 9, 9), dtype=np.int8)
        # define 1 edge
        
        # edge
        for j in range(9):
            for k in range(9):
                if j == 0 or j == 8 or k == 0 or k == 8:
                    out[0, j, k] = 1
         
        # my pieces
        for j in range(9):
            for k in range(9):
                if j > 0 and j < 8 and k > 0 and k < 8:
                    if self.data[j-1, k-1] == turn:
                        out[1, j, k] = 1
        
        # op pieces
        for j in range(9):
            for k in range(9):
                if j > 0 and j < 8 and k > 0 and k < 8:
                    if self.data[j-1, k-1] == -turn:
                        out[2, j, k] = 1
         
        # last move
        if not move is None:               
            out[3, move[0][0]+1, move[0][1]+1] = 1
            out[4, move[1][0]+1, move[1][1]+1] = 1
            
        # whose first
        if turn == -1:
            for j in range(9):
                for k in range(9):
                    out[5, j, k] = 1
        return np.array(out)
    
    def plot(self, is_next_move=False, turn=None):                        # plot the board
        image = self.data.copy()
        if is_next_move:
            if turn not in [-1, 1]:
                raise ValueError("Turn must be -1 or 1, or Must input a turn for next moves")
            else:
                next_moves = self.get_moves(turn)
                if len(next_moves) == 0:
                    raise ValueError("Game is over already")
                next_pos = list(zip(*next_moves))[1]
                for pos in next_pos:
                    image[pos] = turn / 2
        plt.imshow(image, cmap='gray')
        plt.xticks(range(7), range(7))
        plt.yticks(range(7), range(7))
        plt.show()
        
    def get_greedy_move(self, turn, moves=None):
        best_score = -50
        # get all possible moves if not provided
        if moves is None:
            moves, corr_dict = self.get_moves(turn)
            for item in corr_dict:
                moves.append(item)
        
        if len(moves) == 0:
            raise ValueError('No Possible Moves')
        
        best_moves = []
        # calculate greedy move
        for (x0, y0), (x1, y1) in moves:
            tmp_score = 0
            if abs(x0-x1) <= 1 and abs(y0-y1) <= 1:
                tmp_score += 1
            for dr in range(-1, 2):
                for dc in range(-1, 2):
                    try:
                        if x1+dr >= 0 and y1+dc >= 0:
                            tmp_score += self.data[x1+dr, y1+dc] == -turn
                    except:
                        pass
            if tmp_score > best_score:
                best_moves = [((x0, y0), (x1, y1))]
                best_score = tmp_score
            elif tmp_score == best_score:
                best_moves.append(((x0, y0), (x1, y1)))
        return choice(best_moves)
                
    def is_valid(self, turn, pos, get_pos=False):
        r = pos[0]
        c = pos[1]
        if self.data[r, c] != 0:
            if not get_pos:
                return False
            else:
                return
        else:
            for dr in range(-2, 3):
                for dc in range(-2, 3):
                    new_r = r+dr
                    new_c = c+dc
                    if new_r >= 0 and new_c >= 0 and new_r < 7 and new_c < 7 and self.data[new_r, new_c] == turn:
                        if not get_pos:
                            return True
                        else:
                            yield new_r, new_c, dr, dc
            if not get_pos:
                return False
        
    def get_moves(self, turn):
        action_mask = np.zeros(792, dtype=np.int8)
        next_moves = []
        corr_dict = {}
        for r in range(7):
            for c in range(7):
                has_duplicate_move = False      # move within the radius of one of another friendly piece is called
                for new_r, new_c, dr, dc in self.is_valid(turn, (r, c), True): # duplicate move
                    if new_r >= 0 and new_c >= 0 and new_r < 7 and new_c < 7 and self.data[new_r, new_c] == turn:
                        if abs(dr) <= 1 and abs(dc) <=1:
                            if has_duplicate_move: 
                                cur_move = ((new_r, new_c), (r, c))
                                corr_dict[cur_move] = dup_move
                            elif self.data[new_r, new_c] == turn:
                                dup_move = ((new_r, new_c), (r, c))
                                next_moves.append(dup_move) 
                                has_duplicate_move = True
                        elif self.data[new_r, new_c] == turn:
                            cur_move = ((new_r, new_c), (r, c))
                            next_moves.append(cur_move) 
                        else:
                            continue

        return next_moves, corr_dict
        
    def move_to(self, turn, pos0, pos1):
        x0 = pos0[0]
        y0 = pos0[1]
        x1 = pos1[0]
        y1 = pos1[1]
        
        if not self.is_valid(turn, pos1):
            raise ValueError("This move: " + str((pos0, pos1)) + " of turn: " + str(turn) + " is invalid") 
        elif self.data[x0, y0] != turn:
            raise ValueError("The starting position is not your piece")
        else:
            self.data[x1, y1] = turn
            if abs(x0 - x1) > 1 or abs(y0 - y1) > 1:   # jump move
                self.data[x0, y0] = 0

            for dr in range(-1, 2):                  # infection mode!!!!
                for dc in range(-1, 2):
                    if x1+dr >= 0 and y1+dc >= 0 and x1+dr < 7 and y1+dc < 7:
                        if self.data[x1+dr, y1+dc] == -turn:  # convert any piece of the opponent to 'turn'
                            self.data[x1+dr, y1+dc] = turn
    
    def evaluate(self, turn, this_turn, max_score=1, min_score=0.001):
        turn_no=0
        op_no=0
        for r in range(7):
            for c in range(7):
                if self.data[r, c] == turn:
                    turn_no += 1
                elif self.data[r, c] == -turn:
                    op_no += 1
        if len(self.get_moves(this_turn)[0]) == 0:# if one of them can no longer move, count and end
            if turn_no > op_no:
                return max_score
            else:
                return -max_score
        else:
            value = turn_no - op_no
        return value * min_score
    
    @staticmethod    
    def get_manual_q(turn, board):
        '''consider linear growth of win prob with regard to n_diff
        when diff >= 10, the slope grow a bit
        when diff >= 35, consider win prob close to 1 or -1
        ''' 
        turn_no = 0
        op_no = 0
        max1=0.9
        max2=0.95
        # get no diff of turns
        for r in range(7):
            for c in range(7):
                if board[r, c] == turn:
                    turn_no += 1
                elif board[r, c] == -turn:
                    op_no += 1
        diff = turn_no - op_no
        if abs(diff) > 30:
            return diff / abs(diff)
        else:
            return diff / 30
        
        # ignore the rest for now
        sign = diff
        diff = abs(diff)
        if diff < 35:
            diff = (diff / 35) ** 2 * max1
        else:
            diff = max2

        if sign < 0:
            return -diff
        else:
            return diff

In [17]:
class PolicyValueNetwork():
    def __init__(self):
        self._sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=24))
        K.set_session(self._sess)
        
        self._model = load_model('AtaxxZero.h5')
        print("successfully loaded two models")
           
    def predict(self, feature_map, action_mask):        
        return self._sess.run(self._model.outputs, feed_dict={self._model.inputs[0]: feature_map.reshape(-1, 6, 9, 9), \
                                self._model.inputs[1]: action_mask.reshape(-1, 792), K.learning_phase(): 0})

In [32]:
'''These methods are for normal Min max'''
class MinMaxNormal():
    def __init__(self):
        return
    
    @staticmethod
    def evaluate(board, turn):
        turn_no = 0
        op_no = 0
        # get no diff of turns
        for r in range(7):
            for c in range(7):
                if board[r, c] == turn:
                    turn_no += 1
                elif board[r, c] == -turn:
                    op_no += 1
        return (turn_no - op_no)

    @staticmethod
    def is_valid(board, turn, pos):
        r = pos[0]
        c = pos[1]
        if board[r, c] != 0:
            return False
        else:
            for dr in range(-2, 3):
                for dc in range(-2, 3):
                    new_r = r+dr
                    new_c = c+dc
                    if new_r >= 0 and new_c >= 0 and new_r < 7 and new_c < 7 and board[new_r, new_c] == turn:
                        return True
            return False 

    def next_move(self, board, turn):
        next_moves = []
        for r in range(7):
            for c in range(7):
                has_duplicate_move = False      # move within the radius of one of another friendly piece is called
                if self.is_valid(board, turn, (r, c)): # duplicate move
                    for dr in range(-2, 3):
                        for dc in range(-2, 3):
                            new_r = r+dr
                            new_c = c+dc
                            if new_r >= 0 and new_c >= 0 and new_r < 7 and new_c < 7 and board[new_r, new_c] == turn:
                                if abs(dr) <= 1 and abs(dc) <=1:
                                    if board[new_r, new_c] == turn and not has_duplicate_move:
                                        dup_move = ((new_r, new_c), (r, c))
                                        has_duplicate_move = True
                                        yield dup_move
                                elif board[new_r, new_c] == turn:
                                    cur_move = ((new_r, new_c), (r, c))
                                    yield cur_move
                                else:
                                    continue

    def has_next_move(self, board, turn):
        try:
            next(self.next_move(board, turn))
            return True
        except StopIteration:
            return False

    @staticmethod
    def move_to(board, turn, pos0, pos1):
        x0 = pos0[0]
        y0 = pos0[1]
        x1 = pos1[0]
        y1 = pos1[1]

        board = board.copy()
        board[x1, y1] = turn
        if abs(x0 - x1) > 1 or abs(y0 - y1) > 1:   # jump move
            board[x0, y0] = 0

        for dr in range(-1, 2):                  # infection mode!!!!
            for dc in range(-1, 2):
                if x1+dr >= 0 and y1+dc >= 0 and x1+dr < 7 and y1+dc < 7:
                    if board[x1+dr, y1+dc] == -turn:  # convert any piece of the opponent to 'turn'
                        board[x1+dr, y1+dc] = turn
        return board

    def min_max(self, board, turn, target_turn, depth=3, alpha=-100, beta=100, is_max=True, is_root=True):
        '''A recursive alpha beta pruning min_max function
        return: board evaluation, chosen move
        NB. for board evaluation, if the searching was pruned, it will return 100 for a minimizer and -100 for a maximizer'''
        if is_root:
            best_moves = []
        else:
            best_move = ((0, 0), (0, 0))

        if depth == 0 or not self.has_next_move(board, turn): # start to do pruning and selecting once the recursion reaches the end
            result = self.evaluate(board, target_turn)
            return result, None
        else:
            if is_max:
                alpha = -100
            else:
                beta = 100

            for move in self.next_move(board, turn):
                result, _ = self.min_max(self.move_to(board, turn, move[0], move[1]), \
                                    -turn, target_turn, depth-1, alpha, beta, not is_max, False)
                # prun the searching tree or update alpha and beta respectively
                if is_max:
                    if result >= beta:
                        return 100, None
                    elif result > alpha:
                        alpha = result
                        if is_root:
                            best_moves = [move]
                        else:
                            best_move = move
                    elif result == alpha and is_root:
                        best_moves.append(move)
                else:
                    if result <= alpha:
                        return -100, None
                    elif result < beta:
                        beta = result
                        if is_root:
                            best_moves = [move]
                        else:
                            best_move = move
                    elif result == beta and is_root:
                        best_moves.append(move)
            if is_max:
                if is_root:
                    return alpha, choice(best_moves)
                else:
                    return alpha, best_move
            else:
                if is_root:
                    return beta, choice(best_moves)
                else:
                    return beta, best_move

In [33]:
'''These methods are for Min max with ataxxzero'''
class MinMaxZero():
    def __init__(self):
        self._evaluator = PolicyValueNetwork()
        self._policy_dict = self.get_policy_dict()
        
    @staticmethod
    def get_policy_dict():
        '''Get the relation between policy no. and policy'''
        index=0
        policy_dict = {}
        for r in range(7):
            for c in range(7):
                for dr in range(-2, 3):
                    for dc in range(-2, 3):
                        new_r = r + dr
                        new_c = c + dc
                        if (dr != 0 or dc != 0) and (new_r < 7 and new_r >= 0) and (new_c < 7 and new_c >= 0):
                            policy_dict[((r, c), (new_r, new_c))] = index
                            index += 1
        return policy_dict

    @staticmethod
    def get_feature_map(board, turn, pre_move):
        out = np.zeros((6, 9, 9), dtype=np.int8)
        # define 1 edge

        # edge
        for j in range(9):
            for k in range(9):
                if j == 0 or j == 8 or k == 0 or k == 8:
                    out[0, j, k] = 1

        # my pieces
        for j in range(9):
            for k in range(9):
                if j > 0 and j < 8 and k > 0 and k < 8:
                    if board[j-1, k-1] == turn:
                        out[1, j, k] = 1

        # op pieces
        for j in range(9):
            for k in range(9):
                if j > 0 and j < 8 and k > 0 and k < 8:
                    if board[j-1, k-1] == -turn:
                        out[2, j, k] = 1

        # last move
        if not pre_move is None:               
            out[3, pre_move[0][0]+1, pre_move[0][1]+1] = 1
            out[4, pre_move[1][0]+1, pre_move[1][1]+1] = 1

        # whose first
        if turn == -1:
            for j in range(9):
                for k in range(9):
                    out[5, j, k] = 1
        return out
    
    def evaluate(self, feature_map, action_mask, turn, target_turn):
        result = self._evaluator.predict(feature_map, action_mask)
        p = result[0][0]
        if turn == target_turn:
            q = result[1][0]
        else:
            q = -result[1][0]
        return p, q

    @staticmethod
    def is_valid(board, turn, pos):
        r = pos[0]
        c = pos[1]
        if board[r, c] != 0:
            return
        else:
            for dr in range(-2, 3):
                for dc in range(-2, 3):
                    new_r = r+dr
                    new_c = c+dc
                    if new_r >= 0 and new_c >= 0 and new_r < 7 and new_c < 7 and board[new_r, new_c] == turn:
                        yield new_r, new_c, dr, dc
    
    def get_moves(self, board, turn):
        next_moves = []
        dup_moves = []
        action_mask = np.zeros(792, np.int8)
        for r in range(7):
            for c in range(7):
                has_duplicate_move = False      # move within the radius of one of another friendly piece is called
                for new_r, new_c, dr, dc in self.is_valid(board, turn, (r, c)): # duplicate move
                    cur_move = ((new_r, new_c), (r, c))
                    # update action mask
                    action_mask[self._policy_dict[cur_move]] = 1
                    if abs(dr) <= 1 and abs(dc) <=1:
                        if not has_duplicate_move:
                            has_duplicate_move = True
                            next_moves.append(cur_move)
                    else:
                        next_moves.append(cur_move)

        return next_moves, action_mask
    
    @staticmethod
    def move_to(board, turn, pos0, pos1):
        x0 = pos0[0]
        y0 = pos0[1]
        x1 = pos1[0]
        y1 = pos1[1]

        board = board.copy()
        board[x1, y1] = turn
        if abs(x0 - x1) > 1 or abs(y0 - y1) > 1:   # jump move
            board[x0, y0] = 0

        for dr in range(-1, 2):                  # infection mode!!!!
            for dc in range(-1, 2):
                if x1+dr >= 0 and y1+dc >= 0 and x1+dr < 7 and y1+dc < 7:
                    if board[x1+dr, y1+dc] == -turn:  # convert any piece of the opponent to 'turn'
                        board[x1+dr, y1+dc] = turn
        return board

    @staticmethod
    def display_move_prob(move_prob):
        for move, prob in move_prob:
            print(move, prob)
    
    def min_max(self, board, turn, target_turn, depth=3, alpha=-100, beta=100, is_max=True, is_root=True, pre_move=None):
        '''A recursive alpha beta pruning min_max function
        return: board evaluation, chosen move
        NB. for board evaluation, if the searching was pruned, it will return 100 for a minimizer and -100 for a maximizer'''
        if is_root:
            best_moves = []
        else:
            best_move = ((0, 0), (0, 0))

        next_moves, action_mask = self.get_moves(board, turn)
        feature_map = self.get_feature_map(board, turn, pre_move)

        p, q = self.evaluate(feature_map, action_mask, turn, target_turn)

        if depth == 0: # start to do pruning and selecting once the recursion reaches the end
            return q, None
        elif len(next_moves) == 0:
            if (board == target_turn).sum() - (board == -target_turn).sum() > 0:
                return 1, None
            else:
                return -1, None
        else:
            # generate move corresponding p list
            move_prob = []
            all_prob = 0.0
            for move in next_moves:
                prob = p[self._policy_dict[move]]
                move_prob.append((move, prob))
                all_prob += prob
            move_prob = sorted(move_prob, key=lambda x: x[1], reverse=True)

            if is_max:
                alpha = -100
            else:
                beta = 100

            sum_prob = 0.0
            counter = 0
            counter_thresh = len(move_prob) / 15.0
            prob_thresh = (all_prob / len(move_prob)) * 0.8
            # display_move_prob(move_prob)
            for move, prob in move_prob:
                sum_prob += prob
                counter += 1
                # do searching
                try:
                    result, _ = self.min_max(self.move_to(board, turn, move[0], move[1]), \
                                    -turn, target_turn, depth-1, alpha, beta, not is_max, False, move)
                except:
                    print(move)
                    raise
                # prun the searching tree or update alpha and beta respectively
                if is_max:
                    if result >= beta:
                        return 100, None
                    elif result > alpha:
                        alpha = result
                        if is_root:
                            best_moves = [move]
                        else:
                            best_move = move
                    elif result == alpha and is_root:
                        best_moves.append(move)
                else:
                    if result <= alpha:
                        return -100, None
                    elif result < beta:
                        beta = result
                        if is_root:
                            best_moves = [move]
                        else:
                            best_move = move
                    elif result == beta and is_root:
                        best_moves.append(move)

                if sum_prob >= prob_thresh and counter >= counter_thresh:
                    break

            if is_max:
                if is_root:
                    return alpha, choice(best_moves)
                else:
                    return alpha, best_move
            else:
                if is_root:
                    return beta, choice(best_moves)
                else:
                    return beta, best_move

In [42]:
minmax_zero = MinMaxZero()
minmax_normal = MinMaxNormal()
N = 20
N_D = 3
Z_D = 3
zero_win = 0
time_zero = []
time_normal = []
for _ in range(N):
    steps = 0
    a = Ataxx()
    turn = -1
    zero_turn = choice([-1, 1])
    print("zero chose color", zero_turn)
    best_move = None
    while True:
        steps += 1
        #a.plot()
        if turn != zero_turn:
            s = time.time()
            _, best_move = minmax_normal.min_max(a.data, turn, turn, depth=N_D)
            span = time.time() - s
            print("minmax normal, with depth", N_D, "move takes time: ", span)
            if steps > 10 and steps < 60:
                time_normal.append(span)
        else:
            s = time.time()
            _, best_move = minmax_zero.min_max(a.data, turn, turn, depth=Z_D, pre_move=best_move)
            span = time.time() - s
            print("minmax zero, with depth", Z_D, "move takes time: ", span)
            if steps > 10 and steps < 60:
                time_zero.append(span)
        a.move_to(turn, best_move[0], best_move[1])
        turn = -turn
        result = a.evaluate(zero_turn, turn)
        if result == 1:
            print("\n\n\n\n minmax zero win!! \n\n\n\n")
            zero_win += 1
            break
        elif result == -1:
            print("\n\n\n\n minmax normal win!! \n\n\n\n")
            break
print("In the previous ", N, " rounds, zero win ratio is: ", float(zero_win) / float(N))
time_zero = np.array(time_zero)
time_normal = np.array(time_normal)
print("On average, for minmax normal with depth", N_D, \
      ", each move takes time: ", time_normal.mean(),\
      "max time elapsed:", time_normal.max())
print("On average, for minmax zero with depth", Z_D, \
      ", each move takes time: ", time_zero.mean(),\
      "max time elapsed:", time_zero.max())

successfully loaded two models
zero chose color -1
minmax zero, with depth 3 move takes time:  0.4004099369049072
minmax normal, with depth 3 move takes time:  0.15742015838623047
minmax zero, with depth 3 move takes time:  0.05118203163146973
minmax normal, with depth 3 move takes time:  0.40896105766296387
minmax zero, with depth 3 move takes time:  0.07352805137634277
minmax normal, with depth 3 move takes time:  0.4534189701080322
minmax zero, with depth 3 move takes time:  0.06838393211364746
minmax normal, with depth 3 move takes time:  0.36591100692749023
minmax zero, with depth 3 move takes time:  0.0544278621673584
minmax normal, with depth 3 move takes time:  0.4564781188964844
minmax zero, with depth 3 move takes time:  0.08892011642456055
minmax normal, with depth 3 move takes time:  1.069586992263794
minmax zero, with depth 3 move takes time:  0.09245705604553223
minmax normal, with depth 3 move takes time:  0.9402928352355957
minmax zero, with depth 3 move takes time:  0.

minmax normal, with depth 3 move takes time:  2.898340940475464
minmax zero, with depth 3 move takes time:  0.06190800666809082
minmax normal, with depth 3 move takes time:  0.9690120220184326
minmax zero, with depth 3 move takes time:  0.11299395561218262
minmax normal, with depth 3 move takes time:  0.6216781139373779
minmax zero, with depth 3 move takes time:  0.048789024353027344
minmax normal, with depth 3 move takes time:  0.8977499008178711
minmax zero, with depth 3 move takes time:  0.03981614112854004
minmax normal, with depth 3 move takes time:  0.9547429084777832
minmax zero, with depth 3 move takes time:  0.06691813468933105
minmax normal, with depth 3 move takes time:  0.843986988067627
minmax zero, with depth 3 move takes time:  0.0238339900970459
minmax normal, with depth 3 move takes time:  0.9923229217529297
minmax zero, with depth 3 move takes time:  0.05531191825866699
minmax normal, with depth 3 move takes time:  0.5945329666137695
minmax zero, with depth 3 move tak

minmax zero, with depth 3 move takes time:  0.13962578773498535
minmax normal, with depth 3 move takes time:  1.1548941135406494
minmax zero, with depth 3 move takes time:  0.06470513343811035
minmax normal, with depth 3 move takes time:  1.043076992034912
minmax zero, with depth 3 move takes time:  0.11652588844299316
minmax normal, with depth 3 move takes time:  0.9515769481658936
minmax zero, with depth 3 move takes time:  0.06929492950439453
minmax normal, with depth 3 move takes time:  0.9255199432373047
minmax zero, with depth 3 move takes time:  0.04931998252868652
minmax normal, with depth 3 move takes time:  0.8389251232147217
minmax zero, with depth 3 move takes time:  0.051434993743896484
minmax normal, with depth 3 move takes time:  0.6774580478668213
minmax zero, with depth 3 move takes time:  0.048422813415527344
minmax normal, with depth 3 move takes time:  1.3922538757324219
minmax zero, with depth 3 move takes time:  0.06849503517150879
minmax normal, with depth 3 move

minmax zero, with depth 3 move takes time:  0.02305293083190918
minmax normal, with depth 3 move takes time:  0.41611385345458984
minmax zero, with depth 3 move takes time:  0.03494882583618164
minmax normal, with depth 3 move takes time:  0.20758700370788574
minmax zero, with depth 3 move takes time:  0.01293802261352539
minmax normal, with depth 3 move takes time:  0.2014148235321045
minmax zero, with depth 3 move takes time:  0.01394796371459961
minmax normal, with depth 3 move takes time:  0.08751487731933594
minmax zero, with depth 3 move takes time:  0.005984067916870117
minmax normal, with depth 3 move takes time:  0.05422711372375488
minmax zero, with depth 3 move takes time:  0.007838964462280273
minmax normal, with depth 3 move takes time:  0.04014301300048828
minmax zero, with depth 3 move takes time:  0.008770942687988281
minmax normal, with depth 3 move takes time:  0.0040819644927978516
minmax zero, with depth 3 move takes time:  0.003576040267944336




 minmax zero win!

minmax zero, with depth 3 move takes time:  0.07419800758361816
minmax normal, with depth 3 move takes time:  0.7063250541687012
minmax zero, with depth 3 move takes time:  0.08819198608398438
minmax normal, with depth 3 move takes time:  0.6999709606170654
minmax zero, with depth 3 move takes time:  0.06863212585449219
minmax normal, with depth 3 move takes time:  0.6425411701202393
minmax zero, with depth 3 move takes time:  0.0669260025024414
minmax normal, with depth 3 move takes time:  1.220142126083374
minmax zero, with depth 3 move takes time:  0.04554295539855957
minmax normal, with depth 3 move takes time:  1.0427241325378418
minmax zero, with depth 3 move takes time:  0.07981181144714355
minmax normal, with depth 3 move takes time:  0.9980700016021729
minmax zero, with depth 3 move takes time:  0.038851022720336914
minmax normal, with depth 3 move takes time:  0.8640038967132568
minmax zero, with depth 3 move takes time:  0.05653882026672363
minmax normal, with depth 3 move t

...

minmax zero, with depth 3 move takes time:  0.04787707328796387
minmax normal, with depth 3 move takes time:  0.71714186668396
minmax zero, with depth 3 move takes time:  0.03325605392456055
minmax normal, with depth 3 move takes time:  0.586575984954834
minmax zero, with depth 3 move takes time:  0.028709888458251953
minmax normal, with depth 3 move takes time:  0.6758468151092529
minmax zero, with depth 3 move takes time:  0.028053998947143555
minmax normal, with depth 3 move takes time:  0.7370638847351074
minmax zero, with depth 3 move takes time:  0.02520298957824707
minmax normal, with depth 3 move takes time:  0.5302939414978027
minmax zero, with depth 3 move takes time:  0.0231630802154541
minmax normal, with depth 3 move takes time:  0.18035602569580078
minmax zero, with depth 3 move takes time:  0.01958179473876953
minmax normal, with depth 3 move takes time:  0.07721710205078125
minmax zero, with depth 3 move takes time:  0.01510310173034668
minmax normal, with depth 3 move 

minmax zero, with depth 3 move takes time:  0.0647740364074707
minmax normal, with depth 3 move takes time:  1.0600559711456299
minmax zero, with depth 3 move takes time:  0.10510683059692383
minmax normal, with depth 3 move takes time:  0.9401519298553467
minmax zero, with depth 3 move takes time:  0.08402395248413086
minmax normal, with depth 3 move takes time:  0.9334180355072021
minmax zero, with depth 3 move takes time:  0.05450582504272461
minmax normal, with depth 3 move takes time:  0.8670060634613037
minmax zero, with depth 3 move takes time:  0.053067922592163086
minmax normal, with depth 3 move takes time:  0.7309551239013672
minmax zero, with depth 3 move takes time:  0.04753303527832031
minmax normal, with depth 3 move takes time:  1.3522560596466064
minmax zero, with depth 3 move takes time:  0.05227518081665039
minmax normal, with depth 3 move takes time:  0.9432370662689209
minmax zero, with depth 3 move takes time:  0.09227800369262695
minmax normal, with depth 3 move 

minmax zero, with depth 3 move takes time:  0.04359698295593262
minmax normal, with depth 3 move takes time:  0.21213102340698242
minmax zero, with depth 3 move takes time:  0.012309074401855469
minmax normal, with depth 3 move takes time:  0.2016770839691162
minmax zero, with depth 3 move takes time:  0.012589216232299805
minmax normal, with depth 3 move takes time:  0.08778786659240723
minmax zero, with depth 3 move takes time:  0.006425142288208008
minmax normal, with depth 3 move takes time:  0.05708003044128418
minmax zero, with depth 3 move takes time:  0.008181095123291016
minmax normal, with depth 3 move takes time:  0.04247403144836426
minmax zero, with depth 3 move takes time:  0.008564949035644531
minmax normal, with depth 3 move takes time:  0.004155158996582031
minmax zero, with depth 3 move takes time:  0.00606083869934082




 minmax zero win!! 




zero chose color -1
minmax zero, with depth 3 move takes time:  0.06309199333190918
minmax normal, with depth 3 move takes 

minmax zero, with depth 3 move takes time:  0.13701987266540527
minmax normal, with depth 3 move takes time:  0.9808659553527832
minmax zero, with depth 3 move takes time:  0.0543971061706543
minmax normal, with depth 3 move takes time:  0.8740119934082031
minmax zero, with depth 3 move takes time:  0.08088207244873047
minmax normal, with depth 3 move takes time:  0.7430269718170166
minmax zero, with depth 3 move takes time:  0.05429983139038086
minmax normal, with depth 3 move takes time:  1.3494668006896973
minmax zero, with depth 3 move takes time:  0.048246145248413086
minmax normal, with depth 3 move takes time:  0.9456329345703125
minmax zero, with depth 3 move takes time:  0.09356188774108887
minmax normal, with depth 3 move takes time:  0.8399801254272461
minmax zero, with depth 3 move takes time:  0.0754709243774414
minmax normal, with depth 3 move takes time:  1.48335599899292
minmax zero, with depth 3 move takes time:  0.06722402572631836
minmax normal, with depth 3 move tak

minmax zero, with depth 3 move takes time:  0.07076001167297363
minmax normal, with depth 3 move takes time:  0.5144479274749756
minmax zero, with depth 3 move takes time:  0.06798195838928223
minmax normal, with depth 3 move takes time:  0.3617081642150879
minmax zero, with depth 3 move takes time:  0.052787065505981445
minmax normal, with depth 3 move takes time:  0.4608128070831299
minmax zero, with depth 3 move takes time:  0.09377193450927734
minmax normal, with depth 3 move takes time:  1.1396830081939697
minmax zero, with depth 3 move takes time:  0.08826184272766113
minmax normal, with depth 3 move takes time:  0.9512100219726562
minmax zero, with depth 3 move takes time:  0.11576986312866211
minmax normal, with depth 3 move takes time:  0.5996308326721191
minmax zero, with depth 3 move takes time:  0.09281587600708008
minmax normal, with depth 3 move takes time:  1.7768809795379639
minmax zero, with depth 3 move takes time:  0.09814715385437012
minmax normal, with depth 3 move

In [37]:
player = MCTS(c=10, dep_lim=3)

NameError: name 'MCTS' is not defined

In [106]:
game = Ataxx()

In [109]:
%lprun -f player.further_init player.get_next_move(t_lim=3)

In [103]:
player.tester(mm_dep=3, BOTH=True, mode=1, times=10, dep_lim=2, rollout_times=200, verbose=False, t_lim=5)

mm_dep is:  3
####               ####
#### start testing ####


KeyboardInterrupt: 